# Mejorar el rendimiento del modelo ML

## Grid search Keras

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV



# Definir una función para crear el modelo
def create_model(optimizer='adam', init='uniform'):
    model = Sequential()
    model.add(Dense(64, 
                    input_dim=target_train.shape[1], 
                    kernel_initializer=init, 
                    activation='relu'))
    model.add(Dense(32, 
                    kernel_initializer=init, 
                    activation='relu'))
    model.add(Dense(1, 
                    kernel_initializer=init, 
                    activation='sigmoid'))
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, metrics=['accuracy'])
    return model

# Crear el modelo de KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Definir los hiperparámetros para la búsqueda en cuadrícula
param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

# Crear el GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Realizar la búsqueda en cuadrícula
grid_result = grid.fit(features_train, target_train)

# Imprimir los mejores hiperparámetros y el mejor score
print(f"Mejores hiperparámetros: {grid_result.best_params_}")
print(f"Mejor score: {grid_result.best_score_}")

# Evaluar el modelo en el conjunto de prueba
best_model = grid_result.best_estimator_
score = best_model.score(features_test, target_test)
print(f"Score en el conjunto de prueba: {score}")